In [1]:
# import modules
import pandas as pd
 # # # # #

In [2]:
# ratio of radii . Earth to Jupiter. Carole Haswell Book
radius_earth_m = 6.37e6
radius_jupiter_m = 7.15e7
ratio_earth_to_jupiter = radius_earth_m/radius_jupiter_m
ratio_earth_to_jupiter

0.0890909090909091

In [4]:
# Define exoplanet classifications
df_NASA_ranges = pd.DataFrame({"lower":[0,0.5,1.0,1.75,3.5,6.0,14.3],\
                               "upper": [0.5,1.0,1.75,3.5,6.0,14.3,9999],\
                               "classification": ["1.Radius below lower bound","2.Rocky planet","3.Super Earth","4.Sub Neptune","5.Neptune","7.Jupiters","8.Radius above upper bound"] \
                              })
intervals = pd.IntervalIndex.from_arrays(df_NASA_ranges["lower"],df_NASA_ranges["upper"],closed="both")
df_NASA_ranges

,lower,upper,classification
0,0.00,0.50,1.Radius below lower bound
1,0.50,1.00,2.Rocky planet
2,1.00,1.75,3.Super Earth
3,1.75,3.50,4.Sub Neptune
4,3.50,6.00,5.Neptune
5,6.00,14.30,7.Jupiters
6,14.30,9999.00,8.Radius above upper bound


In [67]:
# Define function apply classification based on radius
def classify_exoplanet(wk_radius):
    match = df_NASA_ranges.loc[intervals.contains(wk_radius),"classification"]
    return match.iloc[0] if not match.empty else None

In [68]:
# Define function to clean exoplanet names
def clean_name(s):
    return (
        s.str.lower()
         .str.replace(" ", "", regex=False)
         .str.replace("-", "", regex=False)
         .str.replace("_", "", regex=False)
    )

In [69]:
# load EU data and processed NASA data which is in an excel
df_EU_data = pd.read_csv("/Users/danielbhuglah/Downloads/exoplanet.eu_catalog_02-01-26_16_43_00.CSV")
df_NASA_data = pd.read_excel("NASA_data_with_classifications.xlsx")

In [70]:
# df_EU_data.head(10)

In [71]:
# drop records from EU file that do not have a radius 
# FIXME do I remove this line. Can I take radius from NASA file if I need more candidate Neptunes ?
df_EU_data = df_EU_data[df_EU_data["radius"].notna()]
df_EU_data

,name,planet_status,mass,mass_error_min,mass_error_max,mass_sini,mass_sini_error_min,mass_sini_error_max,radius,radius_error_min,...,star_sp_type,star_age,star_age_error_min,star_age_error_max,star_teff,star_teff_error_min,star_teff_error_max,star_detected_disc,star_magnetic_field,star_alternate_names
0,109 Psc b,Confirmed,5.74300,0.28900,1.01100,6.383,0.078,0.078,1.1520,NaN,...,G5 IV,6.90,0.600,0.600,5600.0,80.000,80.000,NaN,NaN,HD 10697
15,2MASS J0103-5515 (AB)b,Confirmed,10.50000,1.10000,1.10000,NaN,NaN,NaN,1.5400,0.03000,...,M,0.03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Delorme 1, 2MASS J01033563-5515561"
30,51 Eri Ab,Confirmed,4.10000,0.50000,0.50000,NaN,NaN,NaN,1.1100,0.13000,...,F0IV,0.02,0.006,0.006,NaN,NaN,NaN,IR Excess,NaN,NaN
31,51 Peg b,Confirmed,0.47000,0.07000,0.03000,0.460,0.060,0.010,1.9000,0.30000,...,G2IV-V,4.00,2.500,2.500,5758.0,110.623,110.623,NaN,NaN,HD 217014
36,55 Cnc Ae,Confirmed,0.02703,0.00135,0.00135,NaN,NaN,NaN,0.1737,0.00339,...,K0V,10.20,2.500,2.500,5250.0,147.574,147.574,NaN,NaN,"rho1 Cnc, HD 75732, GJ 324 A, rho1 Cnc A, TOI-..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6188,XO-4 b,Confirmed,1.61600,0.10000,0.10100,1.615,0.100,0.100,1.3170,0.02900,...,F5V,2.10,0.600,0.600,5700.0,70.000,70.000,NaN,NaN,NaN
6189,XO-5 b,Confirmed,1.07700,0.03700,0.03700,NaN,NaN,NaN,1.0300,0.05000,...,G8V,8.50,0.800,0.800,5510.0,44.000,44.000,NaN,NaN,NaN
6190,XO-6 b,Confirmed,4.47000,0.12000,0.12000,NaN,NaN,NaN,2.1700,0.20000,...,F5,1.88,0.200,0.900,6720.0,100.000,100.000,NaN,NaN,TYC 4357-995-1
6191,XO-7 b,Confirmed,0.72600,0.03800,0.03800,NaN,NaN,NaN,1.3460,0.02000,...,G0V,1.18,0.710,0.980,6000.0,450.000,250.000,NaN,NaN,BD+85 317


In [72]:
# Covert the eu planet radius (in Jupiter eqivalents) into earth radius equivalents. used to do classifications and normals with NASA archives
df_EU_data["radius_earth"]= df_EU_data["radius"]/ ratio_earth_to_jupiter

In [73]:
# use the earth radius equivalents to assign exoplanet classificaitons
df_EU_data["Exoplanet_class"] = df_EU_data["radius_earth"].apply(classify_exoplanet)
df_EU_data

,name,planet_status,mass,mass_error_min,mass_error_max,mass_sini,mass_sini_error_min,mass_sini_error_max,radius,radius_error_min,...,star_age_error_min,star_age_error_max,star_teff,star_teff_error_min,star_teff_error_max,star_detected_disc,star_magnetic_field,star_alternate_names,radius_earth,Exoplanet_class
0,109 Psc b,Confirmed,5.74300,0.28900,1.01100,6.383,0.078,0.078,1.1520,NaN,...,0.600,0.600,5600.0,80.000,80.000,NaN,NaN,HD 10697,12.930612,7.Jupiters
15,2MASS J0103-5515 (AB)b,Confirmed,10.50000,1.10000,1.10000,NaN,NaN,NaN,1.5400,0.03000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Delorme 1, 2MASS J01033563-5515561",17.285714,8.Radius above upper bound
30,51 Eri Ab,Confirmed,4.10000,0.50000,0.50000,NaN,NaN,NaN,1.1100,0.13000,...,0.006,0.006,NaN,NaN,NaN,IR Excess,NaN,NaN,12.459184,7.Jupiters
31,51 Peg b,Confirmed,0.47000,0.07000,0.03000,0.460,0.060,0.010,1.9000,0.30000,...,2.500,2.500,5758.0,110.623,110.623,NaN,NaN,HD 217014,21.326531,8.Radius above upper bound
36,55 Cnc Ae,Confirmed,0.02703,0.00135,0.00135,NaN,NaN,NaN,0.1737,0.00339,...,2.500,2.500,5250.0,147.574,147.574,NaN,NaN,"rho1 Cnc, HD 75732, GJ 324 A, rho1 Cnc A, TOI-...",1.949694,4.Sub Neptune
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6188,XO-4 b,Confirmed,1.61600,0.10000,0.10100,1.615,0.100,0.100,1.3170,0.02900,...,0.600,0.600,5700.0,70.000,70.000,NaN,NaN,NaN,14.782653,8.Radius above upper bound
6189,XO-5 b,Confirmed,1.07700,0.03700,0.03700,NaN,NaN,NaN,1.0300,0.05000,...,0.800,0.800,5510.0,44.000,44.000,NaN,NaN,NaN,11.561224,7.Jupiters
6190,XO-6 b,Confirmed,4.47000,0.12000,0.12000,NaN,NaN,NaN,2.1700,0.20000,...,0.200,0.900,6720.0,100.000,100.000,NaN,NaN,TYC 4357-995-1,24.357143,8.Radius above upper bound
6191,XO-7 b,Confirmed,0.72600,0.03800,0.03800,NaN,NaN,NaN,1.3460,0.02000,...,0.710,0.980,6000.0,450.000,250.000,NaN,NaN,BD+85 317,15.108163,8.Radius above upper bound


In [74]:
# drop duplicates from eu data panda just in case there are any/
df_EU_data_nodups = df_EU_data.drop_duplicates(subset="name",keep = "first")

In [75]:
# Summarise the exoplanet counts by classificaiton
counts = df_EU_data_nodups.groupby("Exoplanet_class")["name"].nunique()
counts.loc["Total"] = counts.sum()
counts

Exoplanet_class
1.Radius below lower bound       7
2.Rocky planet                 230
3.Super Earth                 1120
4.Sub Neptune                 1954
5.Neptune                      362
7.Jupiters                     749
8.Radius above upper bound     306
Total                         4728
Name: name, dtype: int64

In [76]:
# clean names by removing blanks, dashes and underscors
df_EU_data_nodups["clean_name"] = clean_name(df_EU_data_nodups["name"])
df_NASA_data["clean_name"] = clean_name(df_NASA_data["pl_name"])

In [77]:
# explode the alternate name list from eu data and create one row per exploded name including the orginal data row
df_EU_data_nodups["alt_list"]= df_EU_data_nodups["alternate_names"].str.split(",")
df_EU_data_nodups_exploded = df_EU_data_nodups.explode("alt_list")
df_EU_data_nodups_exploded["clean_alt"] = clean_name(df_EU_data_nodups_exploded["alt_list"])

In [78]:
# setup columns to be pulled in from NASA data
NASA_cols = ["clean_name","ttv_flag"]
# merge the EU data with NASA equivalents - left merge
df_merged_main = df_EU_data_nodups.merge(df_NASA_data,on="clean_name",how="left",suffixes=("","_nasa"))

In [79]:
#df_merged_main

In [80]:
# find rows that failed to match on name field 
df_unmatched = df_merged_main[df_merged_main["pl_name"].isna()]

In [81]:
df_unmatched

,name,planet_status,mass,mass_error_min,mass_error_max,mass_sini,mass_sini_error_min,mass_sini_error_max,radius,radius_error_min,...,pl_pubdate,releasedate,pl_nnotes,st_nphot,st_nrvc,st_nspec,pl_nespec,pl_ntranspec,pl_ndispec,Exoplanet_class_nasa
0,109 Psc b,Confirmed,5.74300,0.28900,1.01100,6.383,0.078,0.078,1.1520,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2MASS J0103-5515 (AB)b,Confirmed,10.50000,1.10000,1.10000,NaN,NaN,NaN,1.5400,0.03000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,51 Eri Ab,Confirmed,4.10000,0.50000,0.50000,NaN,NaN,NaN,1.1100,0.13000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,51 Peg b,Confirmed,0.47000,0.07000,0.03000,0.460,0.060,0.010,1.9000,0.30000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,55 Cnc Ae,Confirmed,0.02703,0.00135,0.00135,NaN,NaN,NaN,0.1737,0.00339,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4711,WD 0310-688 b,Confirmed,3.00000,1.90000,5.50000,NaN,NaN,NaN,1.0300,0.35000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4712,WD 1856+534 Ab,Confirmed,NaN,NaN,NaN,NaN,NaN,NaN,0.9280,0.09000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4713,Wendelstein-1 Ab,Confirmed,0.59000,0.09000,0.10000,NaN,NaN,NaN,1.0000,0.10000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4722,XO-3 Ab,Confirmed,11.79000,0.59000,0.59000,NaN,NaN,NaN,1.2170,0.07300,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [82]:
df_EU_data_nodups_exploded_unmatched = df_EU_data_nodups_exploded[df_EU_data_nodups_exploded["name"].isin(df_unmatched["name"])]

In [83]:
df_merged_alt = df_EU_data_nodups_exploded_unmatched.merge(df_NASA_data,left_on="clean_alt",right_on="clean_name",how="left",suffixes=("","_nasa"))

In [84]:
#df_merged_alt.to_excel("df_merged_alt_TEST.xlsx",index=False)

In [88]:
#df_final = (df_merged_main[~df_merged_main["pl_name"].isna()].append(df_merged_alt).reset_index(drop=TRUE))
df_final = pd.concat([df_merged_main[~df_merged_main["pl_name"].isna()],df_merged_alt],ignore_index=True)

In [89]:
# download the EU data
df_final.to_excel("EU_data_with_classifications_TEST.xlsx",index=False)

In [ ]:
type(df_merged_main)